## 1.0 通过SSH登录设备
通过SSH 登录设备，并通过ssh 获取设备的一些信息。

这次使用的lib 为 [netmiko]。(https://github.com/ktbyers/netmiko)

这是一个开源的python 第三方库。目前支持大量的网络设备。其就是操作网络设备而开发的第三方库。

## 1.1 登录SSH后，获取设备的接口信息

In [1]:
from netmiko import ConnectHandler

In [2]:
iosxr3 = {
    'device_type': 'cisco_xr',
    'ip':   '172.20.3.102',
    'username': 'admin',
    'password': 'admin',
    'port' : 22          # optional, defaults to 22    
}

net_connect = ConnectHandler(**iosxr3)

output = net_connect.send_command('show ip int brief')
print(output)


Thu Aug  9 23:59:21.209 UTC

Interface                      IP-Address      Status          Protocol Vrf-Name
MgmtEth0/0/CPU0/0              172.20.3.102    Up              Up       default 
GigabitEthernet0/0/0/0         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/1         unassigned      Up              Up       default 
GigabitEthernet0/0/0/2         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/3         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/4         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/5         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/6         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/7         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/8         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/9         unassigned      Shutdown        Down     default

## 1.2 修改接口的IP地址信息

In [3]:
interface_cfg = ["interface GigabitEthernet0/0/0/1",
                "no shutdown",
                "commit",
                "end"]

output = net_connect.send_config_set(interface_cfg)
print(output)

config term

Thu Aug  9 05:13:33.476 UTC
RP/0/0/CPU0:xrv2(config)#interface GigabitEthernet0/0/0/1

RP/0/0/CPU0:xrv2(config-if)#no shutdown

RP/0/0/CPU0:xrv2(config-if)#commit

Thu Aug  9 05:13:33.936 UTC
RP/0/0/CPU0:xrv2(config-if)#end

RP/0/0/CPU0:xrv2#


## 1.3 使用jinja2模板
Jinja2 是一个 Python 的功能齐全的模板引擎。它有完整的 unicode 支持，一个可选 的集成沙箱执行环境，被广泛使用，以 BSD 许可证授权。

http://docs.jinkan.org/docs/jinja2/


**通过配置模板，可以把模板文件和数据文件进行分离。（模板文件）+ （数据）生成最终的设备配置。**

数据文件可以是相同的数据结构，这些数据的结构，可以使用yang module 进行定义。定义的yang module 用于数据的格式的校验。





In [62]:
from jinja2 import Environment, FileSystemLoader

In [63]:
# 加载模板的目录。在这个目录中包含了很多的配置模板文件。
env = Environment(loader=FileSystemLoader('./config_template/', 'utf-8'))

In [64]:
print(env.list_templates())

['.ipynb_checkpoints/xrv_hostname-checkpoint.j2', '.ipynb_checkpoints/xrv_interface-checkpoint.j2', 'vios_interace.j2', 'vios_lldp.j2', 'vios_mgt.j2', 'vios_ssh.j2', 'xrv_bgp.j2', 'xrv_hostname.j2', 'xrv_interface.j2', 'xrv_isis.j2', 'xrv_lldp.j2', 'xrv_mgt.j2', 'xrv_sr.j2', 'xrv_ssh.j2']


In [28]:
# 加载一个模板文件，
hostname = env.get_template("xrv_hostname.j2")

In [65]:
#  使用变量渲染模板
print(hostname.render(hostname="xrv1",commit=True))


hostname xrv1


commit


end




In [66]:
import yaml

In [86]:
interfaces = yaml.load(open("./data/xrv1_interfaces.yaml").read())

In [87]:
interfaces

{'commit': True,
 'interfaces': [{'ipaddress': '99.1.2.1', 'name': 'Gi0/0/0/0'},
  {'ipaddress': '99.1.2.5', 'name': 'Gi0/0/0/1'},
  {'ipaddress': '99.1.2.9', 'name': 'Gi0/0/0/2'},
  {'ipaddress': '99.1.2.13', 'name': 'Gi0/0/0/3'},
  {'ipaddress': '99.1.2.17', 'name': 'Gi0/0/0/4'},
  {'ipaddress': '99.1.2.21', 'name': 'Gi0/0/0/5'},
  {'ipaddress': '99.1.2.25', 'name': 'Gi0/0/0/6'}]}

In [88]:
interfaces_temp = env.get_template("xrv_interfaces.j2")

In [89]:
print(interfaces_temp.render(interfaces))



interface Gi0/0/0/0
  ipv4 address 99.1.2.1/30
  no shutdown

interface Gi0/0/0/1
  ipv4 address 99.1.2.5/30
  no shutdown

interface Gi0/0/0/2
  ipv4 address 99.1.2.9/30
  no shutdown

interface Gi0/0/0/3
  ipv4 address 99.1.2.13/30
  no shutdown

interface Gi0/0/0/4
  ipv4 address 99.1.2.17/30
  no shutdown

interface Gi0/0/0/5
  ipv4 address 99.1.2.21/30
  no shutdown

interface Gi0/0/0/6
  ipv4 address 99.1.2.25/30
  no shutdown



commit


end


